In [1]:
import numpy as np
import gym
from IPython.display import clear_output
from time import sleep
import random

env = gym.make("Taxi-v3").env
false = False
true = True

In [2]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [3]:
env.reset()
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [4]:
state = env.encode(3,2,1,0) # Taxi row, taxi column, passenger index, destination index
print("State:", state)

env.s = state
env.render()

State: 344
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [5]:
env.P[state]

{0: [(1.0, 444, -1, False)],
 1: [(1.0, 244, -1, False)],
 2: [(1.0, 344, -1, False)],
 3: [(1.0, 324, -1, False)],
 4: [(1.0, 344, -10, False)],
 5: [(1.0, 344, -10, False)]}

In [6]:
# Solving the environment without reinforcement learning

env.s = state # Set environment to illustration state
illustration_state = state

epochs = 0
penalties, reward = 0,0

frames = [] # For animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    
    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into direct for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
    })
    
    epochs += 1
#end

print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 1750
Penalties incurred: 540


In [7]:
def print_frames(frames):
    for i,frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i+1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
    #end
#end

# print_frames(frames)

In [8]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1
epsilon_decay_rate = 0.001

# For plotting matrices
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()
    
    epochs, penalties, reward, = 0,0,0
    done = False
    
    while not done:
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit action space
        #end
        
        next_state, reward, done, info = env.step(action)
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1-alpha) * old_value + alpha*(reward + gamma*next_max) # Equation
        q_table[state, action]= new_value
        
        if reward == -10:
            penalties += 1
        #end
        
        state = next_state
        epochs +=- 1
    #end
    
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")
    #end
#end

print("Training finished.\n")

Episode: 100000
Training finished.



In [9]:
print(q_table.shape)

(500, 6)


In [10]:
"""Evaluate the agent's performance after Q-learning"""

total_epochs, total_penalties = 0,0
episodes = 100
frames = []

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0,0,0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        
        if reward == -10:
            penalties += 1
        #end
        
        # Put each rendered frame into direct for animation
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
        })
        
        epochs += 1
    #end
    
    total_penalties += penalties
    total_epochs += epochs
#end

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_epochs / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.27
Average penalties per episode: 13.27


In [11]:
print(len(frames))


1327


In [12]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1327
State: 475
Action: 5
Reward: 20
